In [2]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

# 표 크롤링

In [3]:
html = urlopen('http://en.wikipedia.org/wiki/Comparison_of_text_editors')
bs = BeautifulSoup(html, 'html.parser')

table = bs.findAll('table', {'class':'wikitable'})[0]
rows = table.findAll('tr')

In [71]:
#csvFile = open('editors.csv', 'wt+', encoding='UTF-8') 

# w: 쓰기 위해 열기, 파일을 먼저 자른다
# t: 텍스트 모드 (기본값)
# +:업데이트 (읽기 및 쓰기)를 위한 디스크 파일 열기

#writer = csv.writer(csvFile)

df = {}

#try :
colRow = []
for n, row in enumerate(rows) :    
    csvRow = []    
    for cell in row.findAll(['td', 'th']) : # td는 table data, th는 table header
        if n == 0 :
            colRow.append(cell.get_text().strip('\n'))
        else :
            csvRow.append(cell.get_text().strip('\n'))
        #print(csvRow)
    #writer.writerow(csvRow)
    if n == 0 :
        pass
    else :
        df[str(n)] = csvRow
#finally :
#    csvFile.close()

In [72]:
df_t = pd.DataFrame(df).T

In [73]:
df_t.columns = colRow

In [76]:
df_t.columns

Index(['', 'Creator', 'First public release', 'Latest stable version',
       'Latest Release Date', 'Programming language', 'Cost (US$)',
       'Software license', 'Open source', 'CLI available',
       'Minimum installed size'],
      dtype='object')

In [77]:
df_t.sort_values(by='First public release')

,,Creator,First public release,Latest stable version,Latest Release Date,Programming language,Cost (US$),Software license,Open source,CLI available,Minimum installed size
13,ed,Ken Thompson,1970,unchanged from original,,C,Free,?,,,0.04 MB
65,vi,Bill Joy,1976,3.7,,C,Free,BSD or CDDL,,,
64,VEDIT,"Ted Green, Greenview Data",1980,6.24.2,2015‑01‑12,"Assembly, C","$89 standard,239 Pro64",Proprietary,,,0.6 MB
49,Sam,Rob Pike,1980s (early),stable,,C,Free,LPL (OSI approved),,,
26,KEDIT,"Mansfield Software Group, Inc.",1983,1.6.1,2016‑12‑05,C,$129,Proprietary,,,1.1MB
...,...,...,...,...,...,...,...,...,...,...,...
32,Light Table,Chris Granger,2012,0.8.1,2016‑01‑21,ClojureScript,Free,MIT,,,
5,Atom,GitHub,2014,1.46.0,2020‑05‑05,"HTML, CSS, JavaScript, C++",Free,MIT,,,~ 107‑185 MB
12,CudaText,"UVViewSoft (alexey_t, kvichans, matthias030)",2015,1.57.0,2018‑06‑25,Object Pascal (Lazarus),Free,MPL 2.0,,,8 MB Mac\n12 MB Win\n5 Mb *nix
67,Visual Studio Code,Microsoft,2015,1.40.1,2019‑11‑13,"JavaScript, TypeScript",Free,MIT,,,~ 53‑82 MB


## 뉴스 크롤링

In [78]:
import requests

In [150]:
s_word = '강용석'
s_url = requests.get('https://search.naver.com/search.naver?query='+s_word+'&where=news&ie=utf8&sm=nws_hty', 
                     headers={'User-Agent':'Chrome/51.0.2704.103'})
bs = BeautifulSoup(s_url.text, 'html.parser')

# news_list = bs.select('ul.list_news > li') #news_list container 추출
news_links = bs.findAll('a', 'news_tit') # 뉴스 title 부분만 추출, 아예 직접 태그 지정 (link, title 정보 포함)

In [151]:
link_list = []
title_list = []
for i in news_links :
    link_list.append(i.attrs['href'])
    title_list.append(i.attrs['title'])
    
df = pd.DataFrame({'title':title_list, 'link':link_list})
df

,title,link
0,'대통령 명예훼손' 강용석 체포…출석요구 4차례 불응(종합),http://yna.kr/AKR20201208097551004?did=1195m
1,"강용석 자택 체포...가세연 ""文정권, 우파 유튜버 박살내려고""",https://news.joins.com/article/olink/23535270
2,"'강용석 체포'에 가세연 ""김어준은 100번 넘게 체포됐어야""",http://www.mediatoday.co.kr/news/articleView.h...
3,"'허위사실 유포' 혐의 강용석, 8시간 경찰 조사 후 석방(종합)",https://www.nocutnews.co.kr/news/5461466
4,"[속보] '가세연' 강용석, 문대통령 명예훼손 혐의로 체포",http://www.busan.com/view/busan/view.php?code=...
5,"[속보] 경찰, '가세연' 강용석 명예훼손 혐의로 체포",http://www.fnnews.com/news/202012081356377790
6,"체포된 강용석, 8시간 조사받고 석방…""집에 가서 빵 마저 먹어야""",https://biz.chosun.com/site/data/html_dir/2020...
7,"[속보] 경찰, ‘가세연’ 강용석 소장 명예훼손 혐의로 체포",https://www.seoul.co.kr/news/newsView.php?id=2...
8,"""식빵 먹다 체포됐다""…강용석 아내가 울며 보내온 사진 한장",https://news.joins.com/article/olink/23535448
9,'강용석 체포' 항의하는 가세연,https://www.news1.kr/photos/view/?4518662


https://hogni.tistory.com/55
* 1시간 마다 자동 전송 bot 만들기